# Federated PyTorch MNIST Tutorial

In [1]:
#Install dependencies if not already installed
!pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.0 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 7.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 8.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 4.1 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 4.0 MB/s eta 0:00:0000:0100:01
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 4.0 MB/s eta 0:00:0000:0100:01
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 827.0 kB/s eta 0:00:00 0:00:01m
     ━━━━━━━━━━━

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import openfl.native as fx
from openfl.federated import FederatedModel,FederatedDataSet


After importing the required packages, the next step is setting up our openfl workspace. To do this, simply run the `fx.init()` command as follows:

In [3]:
#Setup default workspace, logging, etc.
fx.init('torch_cnn_mnist', log_level='METRIC', log_file='./spam_metric.log')

Creating Workspace Directories
Creating Workspace Templates
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 2.3 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 14.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1
    Uninstalling torchvision-0.15.1:
      Successfully uninstalled torchvision-0.15.1

New workspace directory structure:
workspace
├── .workspace
├── data
│   └── MNIST
│       └── raw
├── src
│   ├── ptmnist_inmemory.py
│   ├── pt_cnn.py
│   ├── mnist_utils.py
│   ├── __init__.py
│   └── __pycache__
│       ├── mnist_utils.cpython-310.pyc
│       └── __init__.cpython-310.pyc
├── logs
│   └── cnn_mnist
│       ├── events.out.tfevents.1675185460.andrea-X299-AORUS-Gaming-3-Pro.4050.0
│       └── events.out.tfevents.1

Now we are ready to define our dataset and model to perform federated learning on. The dataset should be composed of a numpy arrayWe start with a simple fully connected model that is trained on the MNIST dataset. 

In [4]:
def one_hot(labels, classes):
    return np.eye(classes)[labels]

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

train_images,train_labels = trainset.train_data, np.array(trainset.train_labels)
train_images = torch.from_numpy(np.expand_dims(train_images, axis=1)).float()

validset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

valid_images,valid_labels = validset.test_data, np.array(validset.test_labels)
valid_images = torch.from_numpy(np.expand_dims(valid_images, axis=1)).float()
valid_labels = one_hot(valid_labels,10)

/home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [5]:
feature_shape = train_images.shape[1]
classes       = 10

fl_data = FederatedDataSet(train_images,train_labels,valid_images,valid_labels,batch_size=32,num_classes=classes)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 5 * 5, 32)
        self.fc2 = nn.Linear(32, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)
    
optimizer = lambda x: optim.Adam(x, lr=1e-4)

def cross_entropy(output, target):
    """Binary cross-entropy metric
    """
    return F.cross_entropy(input=output,target=target)

Here we can define metric logging function. It should has the following signature described below. You can use it to write metrics to tensorboard or some another specific logging.

In [6]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('./logs/cnn_mnist', flush_secs=5)


def write_metric(node_name, task_name, metric_name, metric, round_number):
    writer.add_scalar("{}/{}/{}".format(node_name, task_name, metric_name),
                      metric, round_number)

In [7]:

#Create a federated model using the pytorch class, lambda optimizer function, and loss function
fl_model = FederatedModel(build_model=Net,optimizer=optimizer,loss_fn=cross_entropy,data_loader=fl_data)

[14:45:47] WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=864580;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/utilities/utils.py\utils.py]8;;\:]8;id=329605;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/utilities/utils.py#172\172]8;;\

The `FederatedModel` object is a wrapper around your Keras, Tensorflow or PyTorch model that makes it compatible with openfl. It provides built in federated training and validation functions that we will see used below. Using it's `setup` function, collaborator models and datasets can be automatically defined for the experiment. 

In [8]:
collaborator_models = fl_model.setup(num_collaborators=2)
collaborators = {'one':collaborator_models[0],'two':collaborator_models[1]}#, 'three':collaborator_models[2]}

[14:46:20] WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=550609;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/utilities/utils.py\utils.py]8;;\:]8;id=465458;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/utilities/utils.py#172\172]8;;\

           WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=296753;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/utilities/utils.py\utils.py]8;;\:]8;id=126832;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/utilities/utils.py#172\172]8;;\

In [9]:
#Original MNIST dataset
print(f'Original training data size: {len(train_images)}')
print(f'Original validation data size: {len(valid_images)}\n')

#Collaborator one's data
print(f'Collaborator one\'s training data size: {len(collaborator_models[0].data_loader.X_train)}')
print(f'Collaborator one\'s validation data size: {len(collaborator_models[0].data_loader.X_valid)}\n')

#Collaborator two's data
print(f'Collaborator two\'s training data size: {len(collaborator_models[1].data_loader.X_train)}')
print(f'Collaborator two\'s validation data size: {len(collaborator_models[1].data_loader.X_valid)}\n')

#Collaborator three's data
#print(f'Collaborator three\'s training data size: {len(collaborator_models[2].data_loader.X_train)}')
#print(f'Collaborator three\'s validation data size: {len(collaborator_models[2].data_loader.X_valid)}')

Original training data size: 60000
Original validation data size: 10000

Collaborator one's training data size: 30000
Collaborator one's validation data size: 5000

Collaborator two's training data size: 30000
Collaborator two's validation data size: 5000



We can see the current plan values by running the `fx.get_plan()` function

In [10]:
 #Get the current values of the plan. Each of these can be overridden
print(fx.get_plan())

{
    "aggregator.settings.best_state_path": "save/torch_cnn_mnist_best.pbuf",
    "aggregator.settings.db_store_rounds": 2,
    "aggregator.settings.init_state_path": "save/torch_cnn_mnist_init.pbuf",
    "aggregator.settings.last_state_path": "save/torch_cnn_mnist_last.pbuf",
    "aggregator.settings.log_metric_callback.template": "src.mnist_utils.write_metric",
    "aggregator.settings.rounds_to_train": 10,
    "aggregator.settings.write_logs": true,
    "aggregator.template": "openfl.component.Aggregator",
    "assigner.settings.task_groups.0.name": "train_and_validate",
    "assigner.settings.task_groups.0.percentage": 1.0,
    "assigner.settings.task_groups.0.tasks.0": "aggregated_model_validation",
    "assigner.settings.task_groups.0.tasks.1": "train",
    "assigner.settings.task_groups.0.tasks.2": "locally_tuned_model_validation",
    "assigner.template": "openfl.component.RandomGroupedAssigner",
    "collaborator.settings.db_store_rounds": 1,
    "collaborator.settings.delta_

Now we are ready to run our experiment. If we want to pass in custom plan settings, we can easily do that with the `override_config` parameter

In [11]:
# Run experiment, return trained FederatedModel

final_fl_model = fx.run_experiment(collaborators, override_config={
    'aggregator.settings.rounds_to_train': 5,
    'aggregator.settings.log_metric_callback': write_metric,
})

/home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/federated/task/runner_pt.py:284: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)
/home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/federated/task/runner_pt.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


[14:47:19] METRIC   Round 0, collaborator one is sending metric for task aggregated_model_validation: acc   0.096800                         ]8;id=198008;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/collaborator/collaborator.py\collaborator.py]8;;\:]8;id=192614;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/collaborator/collaborator.py#415\415]8;;\

           METRIC   Round 0, collaborator validate_agg aggregated_model_validation result acc:      0.096800                                   ]8;id=197430;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=573935;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py#559\559]8;;\

/home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/federated/task/runner_pt.py:458: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


[14:47:22] METRIC   Round 0, collaborator one is sending metric for task train: cross_entropy       0.460455                                 ]8;id=295130;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/collaborator/collaborator.py\collaborator.py]8;;\:]8;id=164606;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/collaborator/collaborator.py#415\415]8;;\

...

           METRIC   Round 4, collaborator validate_local locally_tuned_model_validation result acc: 0.980400                                   ]8;id=265718;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=624607;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py#559\559]8;;\

           METRIC   Round 4, aggregator: train <openfl.interface.aggregation_functions.weighted_average.WeightedAverage object at              ]8;id=285207;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=214419;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py#842\842]8;;\
                    0x7f82cb2b2c10> cross_entropy:    0.063437                                                                                                  

           METRIC   Round 4, aggregator: locally_tuned_model_validation                                                                        ]8;id=674378;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=979448;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py#842\842]8;;\
                    <openfl.interface.aggregation_functions.weighted_average.WeightedAverage object at 0x7f82cb2b2c10> acc:     0.980600                        

           METRIC   Round 4, aggregator: aggregated_model_validation <openfl.interface.aggregation_functions.weighted_average.WeightedAverage  ]8;id=918823;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=554698;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py#842\842]8;;\
                    object at 0x7f82cb2b2c10> acc:        0.980700                                                                                              

           METRIC   Round 4: saved the best model with score 0.980700                                                                          ]8;id=778225;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=928435;file:///home/andrea/anaconda3/envs/openFLenv/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py#858\858]8;;\

In [12]:
#Save final model
final_fl_model.save_native('final_pytorch_model')